<h1> Beispiel: Linie </h1>

In [2]:
# importing the module
import cv2
import numpy as np  
import copy    
import random  
import math

import matplotlib.pyplot as plt
from ipywidgets import Image
from ipywidgets import Output
from ipywidgets import Layout
from ipycanvas import Canvas
from ipyevents import Event

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

from sklearn.linear_model import HuberRegressor, LinearRegression
from sklearn.datasets import make_regression
    
    
    
# initiale Einstellungen für RANSAC und HuberRegressor
maxIter = 10 #maximale Anzahl an Iterationen in RANSAC
schwell = 10 #Schwellwert, bis zu dem ein Punkt als Inlier betrachtet wird (RANSAC)
minConsensusSize = 5 #minimale Größe des Consensus Sets (RANSAC)
huberLoss = 1.1 #huber loss Wert / epsilon
standard = True #zeigt an, ob die Linie ohne RANSAC gezeichnet werden soll
ransac = True #zeigt an, ob die RANSAC Linie gezeichnet werden soll
huber = True #zeigt an, ob die Huber Linie gezeichnet werden soll

#Array von Punkten, die ausgewählt wurden
point_array = np.empty((0,2), int) 



#Funktion, die für einen Punkt testet ob dieser ein Inlier ist 
#True, wenn die Distanz des Punktes zur Linie (gegeben durch m*x+b) kleiner als der Schwellwert ist
def inlier(point, schwell, m, b):
    #finde zwei punkte auf der linie
    p1 = np.array([0,0*m+b])
    p2 = np.array([1,1*m+b])
    #berechne Distanz zwischen point und linie definiert durch p1 und p2
    d = np.linalg.norm(np.cross(p2-p1, point-p1))/np.linalg.norm(p2-p1)
    if d <= schwell:
        return True
    else:
        return False
    
    
#Ermittelt über RANSAC die Parameter einer Geradengleichung, gegeben eine Reihe von Punkten (points) 
#weitere Einstellungen (maxIteration=maximale Anzahl an Iterationen, schwell=Schwellwert bis zu welchem Abstand ein Punkt als Inlier betrachtet wird,
#minConsensusSize=mindestanzahl an Punkten im finalen Consensus Set)    
def ransac_line(points, maxIterations, schwell, minConsensusSize ):
    done = False
    bestNumConsensus = 0 #Anzahl der Punkte im Consensus Set ist zu Beginn 0
    bestConsensusSet = np.zeros((len(points),1)) #Consensus Set enhält am Anfang keine Punkte
    
    while not done:
        maxIterations = maxIterations -1 #Anzahl der verbleibenden Iterationen nimmt mit jedem Durchgang ab
        
        #1)Zwei zufällige Punkte auswählen
        p1 = random.randint(0,len(points)-1)
        p2 = p1
        second_found = False
        #solange einen zweiten zufälligen Punkt wählen, bis dieser nicht dem ersten entspricht
        while not second_found:
            p2 = random.randint(0,len(points)-1)
            if p2 != p1:
                second_found = True
        
        #2)Berechne Modellparameter
        #Matrix A enhält, die x-Werte der beiden Punkte in der ersten Spalte und 1 in der zweiten
        #A = np.array([[points[p1,0],1], [points[p2,0],1] ]) 
        #Vektor b enthält die y-Werte der beiden Punkte
        #b = np.array([[points[p1,1]], [points[p2,1]]])

        #die berechneten Modellparameter x ergeben sich durch A^-1 * b
        #x = np.zeros((2,1))
        #x = np.linalg.inv(A).dot(b)
        
        m = (points[p2,1]-points[p1,1])/(points[p2,0]-points[p1,0])
        b = points[p2,1] - m*points[p2,0]
        x = np.array([[m],[b]])
        
        #3)Finde Consensus Set dieses Modells
        numConsensus = 0
        consensusSet = np.zeros((len(points),1)) #Consensus Set kann maximal alle Punkte enthalten
        for i in range(len(points)):
            if inlier(points[i,:], schwell, x[0,0], x[1,0]): #teste für jeden Punkt ob er ein Inlier ist
                consensusSet[numConsensus] = i #wenn ja, speichere den Index dieses Punktes im Consensus Set
                numConsensus = numConsensus+1
        
        #beinhaltet das Consensus Set genug Punkte, dann kann RANSAC beendet werden
        if numConsensus >= minConsensusSize:
            #print("consensus true: ",numConsensus)
            done = True
            
        #wurde in diesem Durchgang ein größeres Consensus Set gefunden, dann wird das als aktuell Bestes gespeichert
        if numConsensus >= bestNumConsensus:
            bestNumConsensus = numConsensus
            bestConsensusSet = np.copy(consensusSet)
            
        #ist die maximale Anzahl an Iterationen durchlaufen, dann endet RANSAC
        if maxIterations == 0:
            #print("max iter true")
            done = True
        
    #finales Consensus Set gefunden
    #4)Berechne Modell mit finalem Consensus Set / mit allen Punkten im finalen Set
    A = np.ones((bestNumConsensus, 2))
    b = np.zeros((bestNumConsensus,1))
    for i in range(bestNumConsensus):
        A[i,0] = points[int(bestConsensusSet[i]),0]
        b[i,0] = points[int(bestConsensusSet[i]),1]
    x = np.zeros((2,1))
    x = np.linalg.pinv(A).dot(b)
    
    #Ergebnis sind die gefundenen Linien-Parameter (m, b)
    return x[0,0] ,x[1,0]



 
#canvas erstellen
img_height = 400
img_width = 400
canvas = Canvas(width=img_width, height=img_height)
canvas.fill_style = '#FDFDFD'
canvas.fill_rect(0, 0, img_width, img_height)

out = Output()



    
# function to display the coordinates of the points clicked on the image
# and also the fitted lines
#called when new point was added 
@out.capture()
def handle_mouse_down(x, y):
    global img_width
    global img_height
    global standard
    global ransac
    global huber
    
    canvas.fill_style = '#FDFDFD'
    canvas.fill_rect(0, 0, img_width, img_height)
 
    
    #add new point to array 
    global point_array       
    point_array = np.append(point_array, np.array([[x,y]]), axis=0)
        
    #draw all points
    for i in range(0, len(point_array)):
        canvas.fill_style = 'black'
        canvas.fill_circle(point_array[i,0],point_array[i,1],5)
             
         
    #calculate line through current points without RANSAC
    A = np.ones((len(point_array),2))
    b = np.zeros((len(point_array),1))
    for i in range(len(point_array)):
        A[i,0] = point_array[i,0]
        b[i] = point_array[i,1]
    x = np.zeros((2,1))       
    x = np.linalg.pinv(A).dot(b)
        
    
    #draw legend
    canvas.font = '20px sans-serif'
    canvas.fill_style = 'red'
    canvas.fill_text("Standard", 10, 395, max_width=None)
    canvas.fill_style = 'green'
    canvas.fill_text("Ransac", 130, 395, max_width=None)
    canvas.fill_style = 'blue'
    canvas.fill_text("Huber", 250, 395, max_width=None)       
    
    
    if len(point_array) > 1:
        if standard:
            #draw line calculated without RANSAC
            startx = 0
            starty = int(startx*float(x[0])+float(x[1]))
            endx = img_width
            endy = int(endx*float(x[0])+float(x[1]))
            canvas.stroke_style = 'red'
            canvas.stroke_line(startx, starty, endx, endy)
                        
        if ransac:
            #draw line calculated with RANSAC
            global maxIter
            global schwell
            global minConsensusSize
            if minConsensusSize > len(point_array):
                minConsensusSize = len(point_array)
            ransac_a, ransac_b = ransac_line(point_array, maxIter, schwell, minConsensusSize)

            ransac_startx = 0
            ransac_starty = int(ransac_startx*ransac_a+ransac_b)
            ransac_endx = img_width
            ransac_endy = int(ransac_endx*ransac_a+ransac_b)
            canvas.stroke_style = 'green'
            canvas.stroke_line(ransac_startx, ransac_starty, ransac_endx, ransac_endy)
            
        if huber:    
            #draw line calculated with HuberRegressor
            huberX = np.zeros((len(point_array),1))
            for i in range(len(huberX)):
                huberX[i,0] = point_array[i,0]
            huberY = point_array[:,1]

            huber_result = HuberRegressor(alpha=0.0, epsilon=huberLoss).fit(huberX, huberY)
            huber_a = huber_result.coef_
            huber_b = huber_result.intercept_

            huber_startx = 0
            huber_starty = int(huber_startx*huber_a+huber_b)
            huber_endx = img_width
            huber_endy = int(huber_endx*huber_a+huber_b)
            canvas.stroke_style = 'blue'
            canvas.stroke_line(huber_startx, huber_starty, huber_endx, huber_endy)
            


# function to display the coordinates of the points clicked on the image
# and also the fitted lines  
#called when parameters were updated 
def update():
    global img_width
    global img_height
    global standard
    global ransac
    global huber
    
    
    canvas.fill_style = '#FDFDFD'
    canvas.fill_rect(0, 0, img_width, img_height)
 
                
    #draw all points
    for i in range(0, len(point_array)):
        canvas.fill_style = 'black'
        canvas.fill_circle(point_array[i,0],point_array[i,1],5)
             
         
    #calculate line through current points without RANSAC
    A = np.ones((len(point_array),2))
    b = np.zeros((len(point_array),1))
    for i in range(len(point_array)):
        A[i,0] = point_array[i,0]
        b[i] = point_array[i,1]
    x = np.zeros((2,1))       
    x = np.linalg.pinv(A).dot(b)
    
    
    #draw legend
    canvas.font = '20px sans-serif'
    canvas.fill_style = 'red'
    canvas.fill_text("Standard", 10, 395, max_width=None)
    canvas.fill_style = 'green'
    canvas.fill_text("Ransac", 130, 395, max_width=None)
    canvas.fill_style = 'blue'
    canvas.fill_text("Huber", 250, 395, max_width=None)       
        
    
    if len(point_array) > 1:
        if standard:
            #draw line calculated without RANSAC
            startx = 0
            starty = int(startx*float(x[0])+float(x[1]))
            endx = img_width
            endy = int(endx*float(x[0])+float(x[1]))
            canvas.stroke_style = 'red'
            canvas.stroke_line(startx, starty, endx, endy)
                        
        if ransac:
            #draw line calculated with RANSAC
            global maxIter
            global schwell
            global minConsensusSize
            if minConsensusSize > len(point_array):
                minConsensusSize = len(point_array)
            ransac_a, ransac_b = ransac_line(point_array, maxIter, schwell, minConsensusSize)

            ransac_startx = 0
            ransac_starty = int(ransac_startx*ransac_a+ransac_b)
            ransac_endx = img_width
            ransac_endy = int(ransac_endx*ransac_a+ransac_b)
            canvas.stroke_style = 'green'
            canvas.stroke_line(ransac_startx, ransac_starty, ransac_endx, ransac_endy)
            
        if huber:    
            #draw line calculated with HuberRegressor
            huberX = np.zeros((len(point_array),1))
            for i in range(len(huberX)):
                huberX[i,0] = point_array[i,0]
            huberY = point_array[:,1]

            huber_result = HuberRegressor(alpha=0.0, epsilon=huberLoss).fit(huberX, huberY)
            huber_a = huber_result.coef_
            huber_b = huber_result.intercept_

            huber_startx = 0
            huber_starty = int(huber_startx*huber_a+huber_b)
            huber_endx = img_width
            huber_endy = int(huber_endx*huber_a+huber_b)
            canvas.stroke_style = 'blue'
            canvas.stroke_line(huber_startx, huber_starty, huber_endx, huber_endy)
            
               


#make Canvas listen to mouse clicks
canvas.on_mouse_down(handle_mouse_down)
display(out)


#apply changes in parameters
def change_values(hub, rans, stand, s, minCons, maxI, loss):
    global maxIter 
    maxIter = maxI
    global schwell 
    schwell = s
    global minConsensusSize 
    minConsensusSize = minCons
    global huberLoss
    huberLoss = loss
    global standard
    standard = stand
    global ransac
    ransac = rans
    global huber
    huber = hub
    update()
    

#display sliders to update parameters
style = {'description_width': 'initial'}
interact(change_values, hub=widgets.Checkbox(value=True, description="Huber (blue)", disable=False,indent=False), rans=widgets.Checkbox(value=True, description="Ransac (green)", disable=False,indent=False), stand=widgets.Checkbox(value=True, description="Standard (Red)", disable=False,indent=False), s=widgets.FloatSlider(min=0, max=200, step=5, value=schwell, description='(RANSAC) Schwellwert (Abstand zum Modell)', style=style, layout=Layout(width='100%')), minCons=widgets.IntSlider(min=0, max=20, step=1, value=minConsensusSize, description='(RANSAC) Mindestgröße Consensus Set', style=style, layout=Layout(width='100%')), maxI=widgets.IntSlider(min=1, max=100, step=1, value=maxIter, description='(RANSAC) Maximale Anzahl an Iterationen', style=style, layout=Layout(width='100%')), loss=widgets.FloatSlider(min=1.0, max=3, step=0.1, value=huberLoss, description='(HUBER) Loss', style=style, layout=Layout(width='100%')));

canvas
  


Output()

interactive(children=(Checkbox(value=True, description='Huber (blue)', indent=False), Checkbox(value=True, des…

Canvas(height=400, width=400)

In [4]:
<h1> Beispiel: Kreis </h1>

SyntaxError: invalid syntax (Temp/ipykernel_23592/662463538.py, line 1)

In [ ]:
# importing the module
import cv2
import numpy as np  
import copy    
import random    

import matplotlib.pyplot as plt
from ipywidgets import Image
from ipywidgets import Output
from ipycanvas import Canvas
from ipyevents import Event

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

from sklearn.linear_model import HuberRegressor, LinearRegression
from sklearn.datasets import make_regression

from scipy.optimize import leastsq
from scipy.optimize import least_squares
    
    
    
# initiale Einstellungen für RANSAC und HuberRegressor
maxIter = 10 #maximale Anzahl an Iterationen in RANSAC
schwell = 10 #Schwellwert, bis zu dem ein Punkt als Inlier betrachtet wird (RANSAC)
minConsensusSize = 5 #minimale Größe des Consensus Sets (RANSAC)
huberLoss = 1 #huber loss Wert / epsilon
standard = True #zeigt an, ob die Linie ohne RANSAC gezeichnet werden soll
ransac = True #zeigt an, ob die RANSAC Linie gezeichnet werden soll
huber = True #zeigt an, ob die Huber Linie gezeichnet werden soll


#Array von Punkten, die ausgewählt wurden
point_array = np.empty((0,2), int) 






#Funktion, die für einen Punkt testet ob dieser ein Inlier ist 
#True, wenn die Distanz des Punktes zum Kreis kleiner als der Schwellwert ist
def inlier(point, schwell, z1, z2, r):
    #berechne Distanz zwischen point und Kreismitte
    d_zp = np.sqrt((point[0]-z1)**2 + (point[1]-z2)**2)
    #Distanz zwischen point und Kreis
    d = d_zp-r
    
    if d <= schwell:
        return True
    else:
        return False
    

    
def getInitialCircleParameters(points):
    #calculate mean point as center
    m_x = 0
    m_y = 0
    for i in range(len(points)):
        m_x = m_x + points[i][0]
        m_y = m_y + points[i][1]
    m_x = int(m_x/len(points))
    m_y = int(m_y/len(points))
    
    #calculate radius as mean distance from center
    r = 0
    for i in range(len(points)):
        r = r + math.sqrt((points[i][0]-m_x)**2 + (points[i][1]-m_y)**2)
    r = int(r/len(points))  
    
    return r, m_x, m_y
    
    
def circle_residual(params, x, y):
    return params[0]**2 - ((x-params[1])**2 + (y-params[2])**2) # (x-z1)^2 + (x-z2)^2 = r2
    
    
    
#Ermittelt über RANSAC die Parameter einer Kreisgleichung, gegeben eine Reihe von Punkten (points) 
#weitere Einstellungen (maxIteration=maximale Anzahl an Iterationen, schwell=Schwellwert bis zu welchem Abstand ein Punkt als Inlier betrachtet wird,
#minConsensusSize=mindestanzahl an Punkten im finalen Consensus Set)    
def ransac_circle(points, maxIterations, schwell, minConsensusSize ):
    done = False
    bestNumConsensus = 0 #Anzahl der Punkte im Consensus Set ist zu Beginn 0
    bestConsensusSet = np.zeros((len(points),1)) #Consensus Set enhält am Anfang keine Punkte
    
    while not done:
        maxIterations = maxIterations -1 #Anzahl der verbleibenden Iterationen nimmt mit jedem Durchgang ab
        
        #1)drei zufällige Punkte auswählen
        p1 = random.randint(0,len(points)-1)
        p2 = p1
        p3 = p1
        second_found = False
        #solange einen zweiten zufälligen Punkt wählen, bis dieser nicht dem ersten entspricht
        while not second_found:
            p2 = random.randint(0,len(points)-1)
            if p2 != p1:
                second_found = True
        third_found = False
        while not third_found:
            p3 = random.randint(0,len(points)-1)
            if p3 != p1 and p3 != p2:
                third_found = True
        points_subset = np.array([points[p1],points[p2],points[p3]])
        
        #2)Berechne Modellparameter
        init_r , init_z1, init_z2 = getInitialCircleParameters(points)
        #init_r = 100
        #init_z1 = 200
        #init_z2 = 200
        init_guess = np.array([init_r, init_z1, init_z2])
        out = leastsq(circle_residual, init_guess, args=(np.ravel(points_subset[:,0]), np.ravel(points_subset[:,1])))
        
        
        
        #3)Finde Consensus Set dieses Modells
        numConsensus = 0
        consensusSet = np.zeros((len(points),1)) #Consensus Set kann maximal alle Punkte enthalten
        for i in range(len(points)):
            if inlier(points[i,:], schwell, out[0][1], out[0][2], out[0][0]): #teste für jeden Punkt ob er ein Inlier ist
                consensusSet[numConsensus] = i #wenn ja, speichere den Index dieses Punktes im Consensus Set
                numConsensus = numConsensus+1
        
        #beinhaltet das Consensus Set genug Punkte, dann kann RANSAC beendet werden
        if numConsensus >= minConsensusSize:
            #print("consensus true: ",numConsensus)
            done = True
            
        #wurde in diesem Durchgang ein größeres Consensus Set gefunden, dann wird das als aktuell Bestes gespeichert
        if numConsensus >= bestNumConsensus:
            bestNumConsensus = numConsensus
            bestConsensusSet = np.copy(consensusSet)
            
        #ist die maximale Anzahl an Iterationen durchlaufen, dann endet RANSAC
        if maxIterations == 0:
            #print("max iter true")
            done = True
        
    #finales Consensus Set gefunden
    #4)Berechne Modell mit finalem Consensus Set / mit allen Punkten im finalen Set
    points_cons = np.zeros((bestNumConsensus,2))
    for i in range(bestNumConsensus):
        points_cons[i] = points[int(consensusSet[i])] 
    init_r , init_z1, init_z2 = getInitialCircleParameters(points_cons)
        #init_r = 100
        #init_z1 = 200
        #init_z2 = 200
    init_guess = np.array([init_r, init_z1, init_z2])  
    out = leastsq(circle_residual, init_guess, args=(np.ravel(points_cons[:,0]), np.ravel(points_cons[:,1])))
    
    #Ergebnis sind die gefundenen Linien-Parameter (r, z1, z2)
    return out[0][0], out[0][1], out[0][2]



 
#canvas erstellen
img_height = 400
img_width = 400
canvas = Canvas(width=img_width, height=img_height)
canvas.fill_style = '#FDFDFD'
canvas.fill_rect(0, 0, img_width, img_height)

out = Output()



    
# function to display the coordinates of the points clicked on the image
# and also the fitted lines
#called when new point was added 
@out.capture()
def handle_mouse_down(x, y):
    global img_width
    global img_height
    global standard
    global ransac
    global huber
    
    canvas.fill_style = '#FDFDFD'
    canvas.fill_rect(0, 0, img_width, img_height)
 
    
    #add new point to array 
    global point_array       
    point_array = np.append(point_array, np.array([[x,y]]), axis=0)
        
    #draw all points
    for i in range(0, len(point_array)):
        canvas.fill_style = 'black'
        canvas.fill_circle(point_array[i,0],point_array[i,1],5)
             
    #draw legend
    canvas.font = '20px sans-serif'
    canvas.fill_style = 'red'
    canvas.fill_text("Standard", 10, 395, max_width=None)
    canvas.fill_style = 'green'
    canvas.fill_text("Ransac", 130, 395, max_width=None)
    canvas.fill_style = 'blue'
    canvas.fill_text("Huber", 250, 395, max_width=None)            
           
        
    #if enough points
    if len(point_array) > 2:
        
        if standard:
            #calculate circle through current points without RANSAC
            init_r , init_z1, init_z2 = getInitialCircleParameters(point_array)
            #init_r = 100
            #init_z1 = 200
            #init_z2 = 200
            init_guess = np.array([init_r, init_z1, init_z2])
            out = leastsq(circle_residual, init_guess, args=(np.ravel(point_array[:,0]), np.ravel(point_array[:,1])))


            #draw circle calculated without RANSAC
            canvas.stroke_style = 'red'
            canvas.stroke_circle( out[0][1], out[0][2], out[0][0])
                        
        
        if ransac:
            #draw circle calculated with RANSAC
            global maxIter
            global schwell
            global minConsensusSize
            if minConsensusSize > len(point_array):
                minConsensusSize = len(point_array)
            ransac_r, ransac_z1, ransac_z2 = ransac_circle(point_array, maxIter, schwell, minConsensusSize)

            canvas.stroke_style = 'green'
            canvas.stroke_circle( ransac_z1, ransac_z2, ransac_r)
            
        if huber:    
            #draw line calculated with robust least squares
            robust_out = least_squares(circle_residual, init_guess, loss='huber', args=(np.ravel(point_array[:,0]), np.ravel(point_array[:,1])))
            canvas.stroke_style = 'blue'
            canvas.stroke_circle(robust_out.x[1], robust_out.x[2], robust_out.x[0])
            
            
        


# function to display the coordinates of the points clicked on the image
# and also the fitted lines  
#called when parameters were updated 
def update():
    global img_width
    global img_height
    global point_array  
    global standard
    global ransac
    global huber
    
    canvas.fill_style = '#FDFDFD'
    canvas.fill_rect(0, 0, img_width, img_height)
 
                
    #draw all points
    for i in range(0, len(point_array)):
        canvas.fill_style = 'black'
        canvas.fill_circle(point_array[i,0],point_array[i,1],5)
             
    #draw legend
    canvas.font = '20px sans-serif'
    canvas.fill_style = 'red'
    canvas.fill_text("Standard", 10, 395, max_width=None)
    canvas.fill_style = 'green'
    canvas.fill_text("Ransac", 130, 395, max_width=None)
    canvas.fill_style = 'blue'
    canvas.fill_text("Huber", 250, 395, max_width=None)            
        
    #if enough points
    if len(point_array) > 2:
        
        if standard:
            #calculate circle through current points without RANSAC
            init_r , init_z1, init_z2 = getInitialCircleParameters(point_array)
            #init_r = 100
            #init_z1 = 200
            #init_z2 = 200
            init_guess = np.array([init_r, init_z1, init_z2])
            out = leastsq(circle_residual, init_guess, args=(np.ravel(point_array[:,0]), np.ravel(point_array[:,1])))

            #draw line calculated without RANSAC
            canvas.stroke_style = 'red'
            canvas.stroke_circle( out[0][1], out[0][2], out[0][0])
                        
        if ransac:
            #draw circle calculated with RANSAC
            global maxIter
            global schwell
            global minConsensusSize
            if minConsensusSize > len(point_array):
                minConsensusSize = len(point_array)
            ransac_r, ransac_z1, ransac_z2 = ransac_circle(point_array, maxIter, schwell, minConsensusSize)

            canvas.stroke_style = 'green'
            canvas.stroke_circle(ransac_z1, ransac_z2, ransac_r)
            
        if huber:    
            #draw line calculated with robust least squares
            robust_out = least_squares(circle_residual, init_guess, loss='huber', args=(np.ravel(point_array[:,0]), np.ravel(point_array[:,1])))
            canvas.stroke_style = 'blue'
            canvas.stroke_circle(robust_out.x[1], robust_out.x[2], robust_out.x[0])
        
        
               


#make Canvas listen to mouse clicks
canvas.on_mouse_down(handle_mouse_down)
display(out)


#apply changes in parameters
def change_values(hub, rans, stand, s, minCons, maxI, loss):
    global maxIter 
    maxIter = maxI
    global schwell 
    schwell = s
    global minConsensusSize 
    minConsensusSize = minCons
    global huberLoss
    huberLoss = loss
    global standard
    standard = stand
    global ransac
    ransac = rans
    global huber
    huber = hub
    update()
    

#display sliders to update parameters
style = {'description_width': 'initial'}
interact(change_values, hub=widgets.Checkbox(value=True, description="Huber (blue)", disable=False,indent=False), rans=widgets.Checkbox(value=True, description="Ransac (green)", disable=False,indent=False), stand=widgets.Checkbox(value=True, description="Standard (red)", disable=False,indent=False), s=widgets.FloatSlider(min=0, max=100, step=1, value=schwell, description='(RANSAC) Schwellwert (Abstand zum Modell)', style=style, layout=Layout(width='100%')), minCons=widgets.IntSlider(min=0, max=20, step=1, value=minConsensusSize, description='(RANSAC) Mindestgröße Consensus Set', style=style, layout=Layout(width='100%')), maxI=widgets.IntSlider(min=1, max=100, step=1, value=maxIter, description='(RANSAC) Maximale Anzahl an Iterationen', style=style, layout=Layout(width='100%')), loss=widgets.FloatSlider(min=1.0, max=5, step=0.25, value=huberLoss, description='(HUBER) Loss', style=style, layout=Layout(width='100%')));


canvas
  


In [10]:
p1 = random.randint(0,len(points)-1)
p2 = p1
second_found = False
#solange einen zweiten zufälligen Punkt wählen, bis dieser nicht dem ersten entspricht
while not second_found:
    p2 = random.randint(0,len(points)-1)
    if p2 != p1:
        second_found = True
print(p1)
print(p2)

4
0


In [11]:
A = np.array([[points[p1,0],1], [points[p2,0],1] ]) 
#Vektor b enthält die y-Werte der beiden Punkte
b = np.array([[points[p1,1]], [points[p2,1]]])

NameError: name 'points' is not defined